In [34]:
import re
import csv
import tqdm
import pandas as pd

import time
import datetime

from bs4 import BeautifulSoup
from selenium import webdriver as wbd
from selenium.webdriver.common.by import By

# 1. Holiday
- 네이버달력 크롤링하는 방식
    - 한달 단위로 크롤링하며 붉은색(비영업일)에 해당하는 day 추출
    - 토요일은 검은색으로 나타나므로 별도로 추가해줄 것

In [35]:
options = wbd.ChromeOptions()
options.add_argument("headless")
options.add_argument("disable-gpu")
driver = wbd.Chrome("./chromedriver.exe", options=options)

html = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EB%84%A4%EC%9D%B4%EB%B2%84%EB%8B%AC%EB%A0%A5&oquery=%EB%84%A4%E3%85%A3%E3%85%87%EB%B2%84%EB%8B%AC%EB%A0%A5&tqi=i4EPmlprvmZsslbDSkdssssssU0-356245'
driver.get(html)

C:\Users\jjgus\AppData\Local\Temp\ipykernel_24896\4121251317.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wbd.Chrome("./chromedriver.exe", options=options)


In [36]:
# 오늘부터 2000.1.1까지 개월 수 계산
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

# arguments
end_date= datetime.datetime(2023,6,30) # 마지막 날짜
start_date = datetime.datetime(2020,1,1) # 시작 날짜

diff_month = diff_month(end_date, start_date)
diff_month

41

In [37]:
start = time.time()  # 시작 시간 저장
holiday_list = []

for _ in range(diff_month): # argument로 개월 수 입력

    html = BeautifulSoup(driver.page_source, 'lxml')
    txt = html.find_all(class_ = 'cld_day')
    yyyymm = html.find_all('span', class_='_title')[0].get_text()
    print(f'[{yyyymm}]')


    for i, v in enumerate(zip(driver.find_elements(By.CLASS_NAME, 'cld_day'), txt)):
        tmp_list = []
        color = re.sub(r'[^0-9]', '', v[0].value_of_css_property('color'))
        day = v[1].string

        if color == '2312312311' or color == '3636361': # 회색, 검은색은 skip
            continue 
            
        print(f'''idx: {i}, color: {color}, day: {day}''')
        year = int(yyyymm[:4])
        month = int(yyyymm[5:7].lstrip("0"))
        day = int(day.lstrip("0"))
        # print(year, month, day)
        # print(datetime.datetime(year, month, day))
        tmp_list.append(day)
        holiday_list.append(datetime.datetime(year, month, day))
        
    # 한달 전으로 이동
    driver.find_element(By.XPATH, value = '//*[@id="direct_calendar"]/div/div[1]/div[1]/a[2]').click()
    time.sleep(1)
    
print("\n[elapsed time]:", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

[2023.06]
idx: 7, color: 25276781, day: 4
idx: 9, color: 25276781, day: 6
idx: 14, color: 25276781, day: 11
idx: 21, color: 25276781, day: 18
idx: 28, color: 25276781, day: 25
[2023.05]
idx: 5, color: 25276781, day: 5
idx: 7, color: 25276781, day: 7
idx: 14, color: 25276781, day: 14
idx: 21, color: 25276781, day: 21
idx: 27, color: 25276781, day: 27
idx: 28, color: 25276781, day: 28
idx: 29, color: 25276781, day: 29
[2023.04]
idx: 7, color: 25276781, day: 2
idx: 14, color: 25276781, day: 9
idx: 21, color: 25276781, day: 16
idx: 28, color: 25276781, day: 23
idx: 35, color: 25276781, day: 30
[2023.03]
idx: 3, color: 25276781, day: 1
idx: 7, color: 25276781, day: 5
idx: 14, color: 25276781, day: 12
idx: 21, color: 25276781, day: 19
idx: 28, color: 25276781, day: 26
[2023.02]
idx: 7, color: 25276781, day: 5
idx: 14, color: 25276781, day: 12
idx: 21, color: 25276781, day: 19
idx: 28, color: 25276781, day: 26
[2023.01]
idx: 0, color: 25276781, day: 1
idx: 7, color: 25276781, day: 8
idx: 14, 

In [38]:
holiday_list[:5]

[datetime.datetime(2023, 6, 4, 0, 0),
 datetime.datetime(2023, 6, 6, 0, 0),
 datetime.datetime(2023, 6, 11, 0, 0),
 datetime.datetime(2023, 6, 18, 0, 0),
 datetime.datetime(2023, 6, 25, 0, 0)]

---
# 2. Saturday
- 네이버달력에서 토요일은 검색으로 나타나므로 별도로 추가

In [39]:
date_list = pd.date_range(start=start_date, end=end_date, freq='D')
date_list[:5]

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05'],
              dtype='datetime64[ns]', freq='D')

In [157]:
weekend_list = [i for i in date_list if i.weekday() > 4] # 5: 토요일, 6: 일요일
weekend_list[:5]

[Timestamp('2020-01-04 00:00:00', freq='D'),
 Timestamp('2020-01-05 00:00:00', freq='D'),
 Timestamp('2020-01-11 00:00:00', freq='D'),
 Timestamp('2020-01-12 00:00:00', freq='D'),
 Timestamp('2020-01-18 00:00:00', freq='D')]

---
# 3. 병합

## 3-1. preprocess
- datetime으로 병합해주기

In [166]:
def preprocess(list):
#     return [datetime.datetime.strptime(i.strftime("%y-%m-%d"), '%y-%m-%d') for i in list] 
    return [i.strftime("%y-%m-%d") for i in list] 

# strftime: datetime, timestamp -> str, strptime: str -> datetime

In [167]:
holiday_list_prep = preprocess(holiday_list)
weekend_list_prep = preprocess(weekend_list)

In [168]:
len(holiday_list_prep)

228

In [169]:
len(weekend_list_prep)

364

In [170]:
len(holiday_list_prep + weekend_list_prep)

592

In [171]:
len(set(holiday_list_prep + weekend_list_prep))

404

In [172]:
final_list = sorted(set(holiday_list_prep + weekend_list_prep), reverse=True)
final_list

['23-06-25',
 '23-06-24',
 '23-06-18',
 '23-06-17',
 '23-06-11',
 '23-06-10',
 '23-06-06',
 '23-06-04',
 '23-06-03',
 '23-05-29',
 '23-05-28',
 '23-05-27',
 '23-05-21',
 '23-05-20',
 '23-05-14',
 '23-05-13',
 '23-05-07',
 '23-05-06',
 '23-05-05',
 '23-04-30',
 '23-04-29',
 '23-04-23',
 '23-04-22',
 '23-04-16',
 '23-04-15',
 '23-04-09',
 '23-04-08',
 '23-04-02',
 '23-04-01',
 '23-03-26',
 '23-03-25',
 '23-03-19',
 '23-03-18',
 '23-03-12',
 '23-03-11',
 '23-03-05',
 '23-03-04',
 '23-03-01',
 '23-02-26',
 '23-02-25',
 '23-02-19',
 '23-02-18',
 '23-02-12',
 '23-02-11',
 '23-02-05',
 '23-02-04',
 '23-01-29',
 '23-01-28',
 '23-01-24',
 '23-01-23',
 '23-01-22',
 '23-01-21',
 '23-01-15',
 '23-01-14',
 '23-01-08',
 '23-01-07',
 '23-01-01',
 '22-12-31',
 '22-12-25',
 '22-12-24',
 '22-12-18',
 '22-12-17',
 '22-12-11',
 '22-12-10',
 '22-12-04',
 '22-12-03',
 '22-11-27',
 '22-11-26',
 '22-11-20',
 '22-11-19',
 '22-11-13',
 '22-11-12',
 '22-11-06',
 '22-11-05',
 '22-10-30',
 '22-10-29',
 '22-10-23',

# 4.  BussniessDay Classifier 함수화

In [85]:
# 같은 기간동안의 연속적인 날짜 적용
naive_sample = sorted(pd.date_range(start=start_date, end=end_date, freq='D'), reverse=True)
naive_sample_df = pd.DataFrame(naive_sample, columns=['Date'])
naive_sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1277 entries, 0 to 1276
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1277 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 10.1 KB


In [86]:
naive_sample_df.head()

,Date
0,2023-06-30
1,2023-06-29
2,2023-06-28
3,2023-06-27
4,2023-06-26


In [102]:
def BussniessDayClassifier(df, list):
    for idx, v in enumerate(df['Date']):
        if v in list:
            df.drop(idx, inplace=True)
            
    return df

In [103]:
BussniessDayClassifier(naive_sample_df, final_list)

,Date
0,2023-06-30
1,2023-06-29
2,2023-06-28
3,2023-06-27
4,2023-06-26
...,...
1270,2020-01-07
1271,2020-01-06
1274,2020-01-03
1275,2020-01-02


In [101]:
for i in naive_sample_df['Date'][:50]:
    print(i)

2023-06-30 00:00:00
2023-06-29 00:00:00
2023-06-28 00:00:00
2023-06-27 00:00:00
2023-06-26 00:00:00
2023-06-23 00:00:00
2023-06-22 00:00:00
2023-06-21 00:00:00
2023-06-20 00:00:00
2023-06-19 00:00:00
2023-06-16 00:00:00
2023-06-15 00:00:00
2023-06-14 00:00:00
2023-06-13 00:00:00
2023-06-12 00:00:00
2023-06-09 00:00:00
2023-06-08 00:00:00
2023-06-07 00:00:00
2023-06-05 00:00:00
2023-06-02 00:00:00
2023-06-01 00:00:00
2023-05-31 00:00:00
2023-05-30 00:00:00
2023-05-26 00:00:00
2023-05-25 00:00:00
2023-05-24 00:00:00
2023-05-23 00:00:00
2023-05-22 00:00:00
2023-05-19 00:00:00
2023-05-18 00:00:00
2023-05-17 00:00:00
2023-05-16 00:00:00
2023-05-15 00:00:00
2023-05-12 00:00:00
2023-05-11 00:00:00
2023-05-10 00:00:00
2023-05-09 00:00:00
2023-05-08 00:00:00
2023-05-04 00:00:00
2023-05-03 00:00:00
2023-05-02 00:00:00
2023-05-01 00:00:00
2023-04-28 00:00:00
2023-04-27 00:00:00
2023-04-26 00:00:00
2023-04-25 00:00:00
2023-04-24 00:00:00
2023-04-21 00:00:00
2023-04-20 00:00:00
2023-04-19 00:00:00


# 5. 실제데이터 응용

In [143]:
raw_data = pd.read_excel('./Fra krw_bloomberg.xlsx')
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        3488 non-null   datetime64[ns]
 1   1x4         3488 non-null   float64       
 2   Unnamed: 2  0 non-null      float64       
 3   Date.1      3960 non-null   datetime64[ns]
 4   3x6         3959 non-null   float64       
 5   Unnamed: 5  0 non-null      float64       
 6   Date.2      3721 non-null   datetime64[ns]
 7   6x9         3720 non-null   float64       
 8   Unnamed: 8  0 non-null      float64       
 9   Date.3      3814 non-null   datetime64[ns]
 10  9x12        3813 non-null   float64       
dtypes: datetime64[ns](4), float64(7)
memory usage: 340.4 KB


위 예제의 end_date(2020년)기준 맞춰주기

In [123]:
raw_data[:852]

,Date,1x4,Unnamed: 2,Date.1,3x6,Unnamed: 5,Date.2,6x9,Unnamed: 8,Date.3,9x12
0,2023-06-09,3.78,NaN,2023-06-12,NaN,NaN,2023-06-12,NaN,NaN,2023-06-12,NaN
1,2023-06-08,3.78,NaN,2023-06-09,3.83,NaN,2023-06-09,3.78,NaN,2023-06-09,3.60
2,2023-06-07,3.77,NaN,2023-06-08,3.85,NaN,2023-06-08,3.80,NaN,2023-06-08,3.68
3,2023-06-05,3.78,NaN,2023-06-07,3.80,NaN,2023-06-07,3.75,NaN,2023-06-07,3.58
4,2023-06-02,3.77,NaN,2023-06-05,3.82,NaN,2023-06-05,3.79,NaN,2023-06-05,3.60
...,...,...,...,...,...,...,...,...,...,...,...
847,2020-01-07,1.49,NaN,2020-01-08,1.40,NaN,2020-01-08,1.33,NaN,2020-01-08,1.28
848,2020-01-06,1.52,NaN,2020-01-07,1.40,NaN,2020-01-07,1.28,NaN,2020-01-07,1.25
849,2020-01-03,1.55,NaN,2020-01-06,1.38,NaN,2020-01-06,1.27,NaN,2020-01-06,1.22
850,2020-01-02,1.53,NaN,2020-01-03,1.37,NaN,2020-01-03,1.29,NaN,2020-01-03,1.26


In [124]:
raw_data = raw_data[:852]

In [125]:
raw_data14 = raw_data[['Date', '1x4']]
raw_data36 = raw_data[['Date.1', '3x6']]
raw_data69 = raw_data[['Date.2', '6x9']]
raw_data912 = raw_data[['Date.3', '9x12']]

In [141]:
print(f'''raw_data14: {len(raw_data14)}
raw_data36: {len(raw_data36)}
raw_data69: {len(raw_data69)}
raw_data912: {len(raw_data912)}''')

raw_data14: 852
raw_data36: 852
raw_data69: 852
raw_data912: 852


In [142]:
BussniessDayClassifier(raw_data14, final_list)

,Date,1x4
0,2023-06-09,3.78
1,2023-06-08,3.78
2,2023-06-07,3.77
3,2023-06-05,3.78
4,2023-06-02,3.77
...,...,...
847,2020-01-07,1.49
848,2020-01-06,1.52
849,2020-01-03,1.55
850,2020-01-02,1.53
